In [17]:
import pyspark
from pyspark.sql import SparkSession
import gzip
import shutil
import pandas as pd
from pyspark.sql import types

In [3]:
spark = SparkSession.builder.master("local[*]").appName("test").getOrCreate()

In [4]:

!wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhvhv/fhvhv_tripdata_2021-01.csv.gz

^C


In [5]:
with gzip.open("fhvhv_tripdata_2021-01.csv.gz", 'rb') as f_in:
    with open("fhvhv_tripdata_2021-01.csv", 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)


In [6]:
df = spark.read.csv("fhvhv_tripdata_2021-01.csv", header=True, inferSchema=True)
df.show()
df.count()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02682|2021-01-01 00:33:44|2021-01-01 00:49:07|         230|         166|   NULL|
|           HV0003|              B02682|2021-01-01 00:55:19|2021-01-01 01:18:21|         152|         167|   NULL|
|           HV0003|              B02764|2021-01-01 00:23:56|2021-01-01 00:38:05|         233|         142|   NULL|
|           HV0003|              B02764|2021-01-01 00:42:51|2021-01-01 00:45:50|         142|         143|   NULL|
|           HV0003|              B02764|2021-01-01 00:48:14|2021-01-01 01:08:42|         143|          78|   NULL|
|           HV0005|              B02510|2021-01-01 00:06:59|2021-01-01 00:43:01|

11908468

In [7]:
df.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', TimestampType(), True), StructField('dropoff_datetime', TimestampType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('SR_Flag', IntegerType(), True)])

In [12]:
df_pandas = pd.read_csv("fhvhv_tripdata_2021-01.csv")

In [14]:
df_pandas.dtypes

hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

In [15]:
spark.createDataFrame(df_pandas).schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('SR_Flag', DoubleType(), True)])

In [28]:
schema = types.StructType([
         types.StructField("hvfhs_license_num", types.StringType(), True),
         types.StructField("dispatching_base_num", types.StringType(), True),
         types.StructField("pickup_datetime", types.TimestampType(), True),
         types.StructField("dropoff_datetime", types.TimestampType(), True),
         types.StructField("PULocationID", types.IntegerType(), True),
         types.StructField("DOLocationID", types.IntegerType(), True),
         types.StructField("SR_Flag", types.IntegerType(), True)
         
         
         ]
    )

In [29]:
df= spark.read.option("header", True).schema(schema).csv("fhvhv_tripdata_2021-01.csv")

In [33]:
df.show(10)

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02682|2021-01-01 00:33:44|2021-01-01 00:49:07|         230|         166|   NULL|
|           HV0003|              B02682|2021-01-01 00:55:19|2021-01-01 01:18:21|         152|         167|   NULL|
|           HV0003|              B02764|2021-01-01 00:23:56|2021-01-01 00:38:05|         233|         142|   NULL|
|           HV0003|              B02764|2021-01-01 00:42:51|2021-01-01 00:45:50|         142|         143|   NULL|
|           HV0003|              B02764|2021-01-01 00:48:14|2021-01-01 01:08:42|         143|          78|   NULL|
|           HV0005|              B02510|2021-01-01 00:06:59|2021-01-01 00:43:01|

In [36]:
df = df.repartition(24)

In [38]:
df.write.parquet('fhvhv/2021/01/')

In [39]:
df = spark.read.parquet('fhvhv/2021/01/')

In [41]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0005|              B02510|2021-01-01 21:12:43|2021-01-01 21:22:10|          42|         243|   NULL|
|           HV0003|              B02765|2021-01-01 13:31:31|2021-01-01 13:41:50|         225|          37|   NULL|
|           HV0005|              B02510|2021-01-02 07:53:35|2021-01-02 08:16:24|         130|         265|   NULL|
|           HV0005|              B02510|2021-01-02 18:40:02|2021-01-02 19:03:22|          85|         177|   NULL|
|           HV0003|              B02765|2021-01-01 14:55:01|2021-01-01 15:05:32|         232|         114|   NULL|
|           HV0003|              B02866|2021-01-03 00:27:18|2021-01-03 00:36:45|

In [42]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: integer (nullable = true)



In [44]:
df.select('pickup_datetime', 'dropoff_datetime','PULocationID', 'DOLocationID').filter(df.hvfhs_license_num == 'HV0003').show(5)

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-------------------+-------------------+------------+------------+
|2021-01-01 13:31:31|2021-01-01 13:41:50|         225|          37|
|2021-01-01 14:55:01|2021-01-01 15:05:32|         232|         114|
|2021-01-03 00:27:18|2021-01-03 00:36:45|          14|          22|
|2021-01-01 01:34:17|2021-01-01 01:49:01|         172|         221|
|2021-01-01 13:05:55|2021-01-01 13:19:06|         250|         254|
+-------------------+-------------------+------------+------------+
only showing top 5 rows



In [45]:
from pyspark.sql import functions as F

In [49]:
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

In [51]:
crazy_stuff('B02884')

's/b44'

In [52]:
crazy_stuff_udf = F.udf(crazy_stuff, returnType=types.StringType())

In [ ]:
df.withColumn('pickup_datetime', F.to_date(df.pickup_datetime)) \
  .withColumn('dropoff_datetime', F.to_date(df.dropoff_datetime)) \
  .withColumn('base_id', crazy_stuff_udf(df.dispatching_base_num)) \
  .select('base_id','pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID') \
  .show()


+---------------+----------------+------------+------------+
|pickup_datetime|dropoff_datetime|PULocationID|DOLocationID|
+---------------+----------------+------------+------------+
|     2021-01-01|      2021-01-01|          42|         243|
|     2021-01-01|      2021-01-01|         225|          37|
|     2021-01-02|      2021-01-02|         130|         265|
|     2021-01-02|      2021-01-02|          85|         177|
|     2021-01-01|      2021-01-01|         232|         114|
|     2021-01-03|      2021-01-03|          14|          22|
|     2021-01-01|      2021-01-01|         172|         221|
|     2021-01-01|      2021-01-01|         250|         254|
|     2021-01-02|      2021-01-02|         186|         265|
|     2021-01-01|      2021-01-01|          79|          25|
|     2021-01-02|      2021-01-02|          17|         225|
|     2021-01-02|      2021-01-02|          49|          21|
|     2021-01-02|      2021-01-02|          39|         218|
|     2021-01-03|      2